## Support Vector Machines（支持向量机, SVM）

支持向量机（SVM）是一组用于分类， 回归和异常值检测的监督学习方法。
支持向量机的优点是：
在高维空间有效。
在维度数量大于样本数量的情况下仍然有效。
在决策功能（称为支持向量）中使用训练点的子集，因此它也是内存有效的。
多功能：可以为决策功能指定不同的内核函数。提供通用内核，但也可以指定自定义内核。
支持向量机的缺点包括：
如果特征数量远远大于样本数量，则该方法可能会导致较差的性能。
支持向量机不直接提供概率估计，这些是使用昂贵的五-折交叉验证计算的（参见下面的分数和概率）。
scikit学习中的支持向量机支持密集（numpy.ndarray可转换为numpy.asarray）和稀疏（任意scipy.sparse）样本向量作为输入。然而，为了使用SVM对稀疏数据进行预测，它必须符合这些数据。为了获得最佳性能，请使用C-ordered numpy.ndarray（密集）或 scipy.sparse.csr_matrix（稀疏）dtype=float64。


## 分类
SVC，NuSVC并且LinearSVC是能够对数据集执行多类分类的类。

In [2]:
from sklearn import svm
X = [[0, 0], [1, 1]]
y=[0,1]
clf=svm.SVC()
clf.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [3]:
clf.predict([[2., 2.]])

array([1])

In [4]:
clf.support_vectors_#获取支持向量

array([[ 0.,  0.],
       [ 1.,  1.]])

In [5]:
clf.support_#获取支持向量的索引

array([0, 1], dtype=int32)

In [6]:
clf.n_support_#获取支持向量每个分类的ｎｕmber

array([1, 1], dtype=int32)

## 多类分类
SVC并NuSVC实施“一对一”的方法（Knerr等，1990）进行多类分类。如果n_class是类的数量，那么构造 n_class * (n_class - 1) / 2 【补充说明：两两组合，可以百度查询：组合数公式】 个分类器，并且从两个类中构建每个训练数据。 为了提供与其他分类器的一致的界面，decision_function_shape选项允许将“一对一”分类器的结果聚合到shape  (n_samples, n_classes) 的决策函数：

In [9]:
X = [[0], [1], [2], [3]]
Y = [0, 1, 2, 3]
clf=svm.SVC(decision_function_shape='ovo')
clf.fit(X,Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
dec=clf.decision_function([[1]])

In [11]:
dec.shape

(1, 6)

另一方面，LinearSVC实施“一对一”的多类策略，从而训练n_class模型。如果只有2类，只有一个模型被训练：

In [13]:
lin_clf = svm.LinearSVC()
lin_clf.fit(X, Y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
lin_dec=lin_clf.decision_function([[1]]).shape[1]
print(lin_dec)

4


有关决策函数的完整描述，请参阅数学公式。
请注意，LinearSVC它还实现了一个替代的多类策略，即由Crammer和Singer制定的所谓的多类SVM，通过使用该选项multi_class='crammer_singer'。这种方法是一致的，这对于一对其余分类是不正确的。在实践中，一对其余分类通常是首选的，因为结果大部分是相似的，但运行时间明显较少。

对于“一对其余” LinearSVC，属性coef_和intercept_分别具有形状[n_class，n_features]和[n_class]。 系数的每一行对应于n_class中的一个，许多“一对其余”分类器，并且类似于截距，按照“一”类的顺序。

在“一对一”SVC的情况下，属性的布局有一点更多的参与。 在具有线性内核的情况下，coef_和intercept_的布局类似于上述描述的LinearSVC，除了coef_的形状是[n_class * (n_class - 1) / 2, n_features]，对应于 许多二进制分类器。0到n级的顺序为“0 vs 1”，“0 vs 2”，...“0 vs n”，“1 vs 2”，“1 vs 3”，“1 vs n” 。。“n-1 vs n”。
 
double_coef_的形状是[n_class-1，n_SV]，有点难以把握布局。 列对应于任何 n_class * (n_class - 1) / 2 “一对一”分类器中涉及的支持向量。 每个支持向量在n_class - 1分类器中使用。 每行中的n_class - 1个条目对应于这些分类器的双重系数。

## 不平衡问题
在需要更加重视某些类别或某些个人样本关键字class_weight并 sample_weight可以使用的问题中。
SVC（但不是NuSVC）class_weight在fit方法中实现一个关键字 。它的形式的字典 ，其中值是浮点数> 0，则设置参数类的到。{class_label : value}Cclass_labelC * value

## 回归
支持向量分类的方法可以扩展到解决回归问题。这种方法称为支持向量回归。
由支持向量分类产生的模型（如上所述）仅取决于训练数据的一部分，因为用于构建模型的成本函数不在于超出边际的训练点。类似地，由支持向量回归产生的模型仅取决于训练数据的子集，因为构建模型的成本函数忽略了接近模型预测的任何训练数据。
有支持向量回归的三种不同的实现方式： SVR，NuSVR和LinearSVR。LinearSVR 提供比SVR更快的实现，但只考虑线性内核，而NuSVR实现与SVR和LinearSVR略有不同的方式。有关详细信息，请参阅 实施细节。
与分类类一样，拟合方法将作为参数向量X，y，只有在这种情况下，y预期具有浮点值而不是整数值：

In [17]:
from sklearn import svm

In [18]:
X = [[0, 0], [2, 2]]
y = [0.5, 2.5]
clf = svm.SVR()
clf.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [19]:
clf.predict([[1,1]])

array([ 1.5])

## 密度估计，新奇检测
一类SVM用于新颖性检测，即给定一组样本，它将检测该集合的软边界，以便将新点归类为属于该集合。调用实现这个的类OneClassSVM。
在这种情况下，由于它是一种无监督的学习方法，拟合方法将仅作为输入数组X，因为没有类标签。
有关此用法的更多详细信息，请参阅新颖性和异常值检测。

## 实用窍门
避免数据复制：对SVC，SVR，NuSVC和 NuSVR，如果通过一定的方法将数据不是C有序连续的，而双精度，它会调用底层的C实现之前复制。您可以通过检查其flags属性来检查给定的numpy数组是否为C 连续的。
对于LinearSVC（和LogisticRegression）作为numpy数组传递的任何输入将被复制并转换为liblinear内部稀疏数据表示（非零组件的双精度浮点数和int32索引）。如果你想要适合一个大规模的线性分类器，而不需要复制一个密集的数字C连续的双精度数组作为输入，我们建议使用SGDClassifier该类。目标函数可以配置为与LinearSVC 模型几乎相同。
内核缓存大小：对SVC，SVR，nuSVC和 NuSVR，内核缓存的大小有较大的问题，在运行时间有很大的影响。如果您有足够的RAM可用，建议设置cache_size为比默认值200（MB）更高的值，例如500（MB）或1000（MB）。
设置为C：C是1在默认情况下，这是一个合理的默认选择。如果你有很多嘈杂的观察结果，你应该减少它。它对应于更多的估计。
支持向量机算法不是尺度不变量，因此强烈建议您扩展数据。例如，将输入向量X上的每个属性缩放到[0,1]或[-1，+ 1]，或将其标准化为平均值0和方差1.注意，必须将相同的缩放应用于测试向量获得有意义的结果。有关缩放和归一化的更多详细信息，请参见 预处理数据。
参数nu在NuSVC/ OneClassSVM/ NuSVR 近似训练误差和支持向量的分数。
在SVC，如果用于分类的数据不平衡（例如，许多正数和少量负数），请设置class_weight='balanced'和/或尝试不同的惩罚参数C。
底层LinearSVC实现使用随机数生成器来在拟合模型时选择特征。因此，相同输入数据的结果略有不同，因此并不罕见。如果发生这种情况，请尝试使用较小的tol参数。
使用提供的L1惩罚产生稀疏解，即只有特征权重的子集不同于零，并有助于决策函数。增加产量是一个更复杂的模型（选择更多的特征）。可以使用产生“零”模型（所有权重等于零）的值。LinearSVC(loss='l2', penalty='l1', dual=False)CCl1_min_c